In [ ]:
from source_files import *

import math
import numpy as np
import pandas as pd
import numba

In [ ]:
sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

@numba.vectorize
def snow_volume(snow_depth, x_resolution, y_resolution):
    if snow_depth < 0:
        return 0
    else:
        return math.fabs(x_resolution) * math.fabs(y_resolution) * snow_depth

In [ ]:
df = pd.DataFrame({
        'aso_snow_depth': aso_snow_depth_values.ravel(),
        'sfm_snow_depth': sfm_snow_depth_values.ravel(),
        'sd_difference': sd_difference_values.ravel(),
        'aso_snow_volume': snow_volume(
            aso_snow_depth_values.ravel(), 
            aso_snow_depth.x_resolution, 
            aso_snow_depth.y_resolution
        ),
        'sfm_snow_volume': snow_volume(
            sfm_snow_depth_values.ravel(), 
            sfm_snow_depth.x_resolution, 
            sfm_snow_depth.y_resolution
        ),
        'casi_class': casi_classification.ravel(),
    },
    dtype='float32'
)

In [ ]:
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

positive_sfm = df.query('sfm_snow_depth >= 0')
negative_sfm = df.query('sfm_snow_depth < 0')

## Median values by CASI classes

In [ ]:
columns = ['casi_class', 'aso_snow_depth', 'sfm_snow_depth']
table_style = "{:.2f} m"

### All values

In [ ]:
print(f"ASO Median depth {df['aso_snow_depth'].median():.4f}m")
print(f"SfM Median depth {df['sfm_snow_depth'].median():.4f}m")

In [ ]:
df[columns].groupby('casi_class').median().style.format(table_style)

### Positive SfM values

In [ ]:
print(f"ASO Median depth {positive_sfm['aso_snow_depth'].median():.4f}m")
print(f"SfM Median depth {positive_sfm['sfm_snow_depth'].median():.4f}m")

In [ ]:
positive_sfm[columns].groupby('casi_class').median().style.format(table_style)

### Negative SfM values

In [ ]:
negative_sfm[columns].groupby('casi_class').median().style.format(table_style)

## Mean values by CASI classes

### All values

In [ ]:
print(f"ASO Mean depth {df['aso_snow_depth'].mean():.4f}m")
print(f"SfM Mean depth {df['sfm_snow_depth'].mean():.4f}m")

In [ ]:
df[columns].groupby('casi_class').mean().style.format(table_style)

### Positive SfM values

In [ ]:
print(f"ASO Mean depth {positive_sfm['aso_snow_depth'].mean():.4f}m")
print(f"SfM Mean depth {positive_sfm['sfm_snow_depth'].mean():.4f}m")

In [ ]:
positive_sfm[columns].groupby('casi_class').mean().style.format(table_style)

### Negative SfM values

In [ ]:
negative_sfm[columns].groupby('casi_class').mean().style.format(table_style)

# Snow Volume

Volume for negative SfM snow depth values was not calculated.

In [ ]:
table_style = {
    'sfm_snow_volume': "{:,.0f} m<sup>3</sup>",
    'aso_snow_volume': "{:,.0f} m<sup>3</sup>",
    'aso - sfm': "{:,.0f} m<sup>3</sup>",
    'sfm % to aso': "{:.2%}",
}

In [ ]:
total_volume = df[['sfm_snow_volume', 'aso_snow_volume']].sum()
total_volume = pd.DataFrame({
    'sfm_snow_volume': [total_volume.sfm_snow_volume],
    'aso_snow_volume': [total_volume.aso_snow_volume],
})
total_volume['aso - sfm'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

In [ ]:
total_volume = df[['sfm_snow_volume', 'aso_snow_volume', 'casi_class']].groupby('casi_class').sum()
total_volume['aso - sfm'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

## Positive SfM values

In [ ]:
total_volume = positive_sfm[['sfm_snow_volume', 'aso_snow_volume']].sum()
total_volume = pd.DataFrame({
    'sfm_snow_volume': [total_volume.sfm_snow_volume],
    'aso_snow_volume': [total_volume.aso_snow_volume],
})
total_volume['aso - sfm'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)